# Data Wrangling in Pandas

This session draws primarily on Chapter 7 in Python for Data Analysis.  It covers methods that are used heavily in 'data wrangling', which refers to the data manipulation that is often needed to transform raw data into a form that is useful for analysis.  We'll stick to the data and examples used in the book for most of this session, since the examples are clearer on the tiny datasets.  After that we will work through some of these methods again using real data.

Key methods covered include:

* Merging and Concatenating
* Reshaping data
* Data transformations
* Categorization
* Detecting and Filtering Outliers
* Creating Dummy Variables


In [2]:
import pandas as pd
import numpy as np

## Merging

Merging two datasets is a very common operation in preparing data for analysis.  It generally means adding columns from one table to colums from another, where the value of some key, or merge field, matches.

Let's begin by creating two simple DataFrames to be merged.

In [3]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


Here is a many to one merge.  The join field is implicit, based on what columns it finds in common between the two dataframes. Note that they share some values of the key field (a, b), but do not share key values c and d.  What do you expect to happen when we merge them? The result contains the values from both inputs where they both have a value of the merge field, which is 'key' in this example.  The default behavior is that the key value has to be in both inputs to be kept.  In set terms it would be an intersection of the two sets.

In [4]:
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


Here is the same merge, but making the join field explicit.


In [5]:
pd.merge(df1,df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [6]:
#what if there are more than one value of key in both dataframes? This is a many-to-many merge.
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df3 = pd.DataFrame({'key': ['a', 'b', 'b', 'd'],'data2': range(4)})
print(df1)
print(df3)
pd.merge(df1,df3, on='key')
#This produces a cartesian product of the number of occurrences of each key value in both dataframes:
# (b shows up 3 times in df1 and 2 times in df3, so we get 6 occurrences in the result of the merge)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   b
3      3   d


,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,6,b,1
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0


In [7]:
# There are several types of joins: left, right, inner, and outer. Let's compare them.
# How does a 'left' join compare to our initial join?  Note that it keeps the result if it shows up in df1,
# regardless of whether it also shows up in df2.  It fills in a value of NaN for the missing value from df2.
pd.merge(df1,df3, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,2.0
2,1,b,1.0
3,1,b,2.0
4,2,a,0.0
5,3,c,NaN
6,4,a,0.0
7,5,a,0.0
8,6,b,1.0
9,6,b,2.0


In [8]:
#How does a 'right' join compare?  Same idea, but this time it keeps a result if it shows up in df2, regardless
# of whether it also shows up in df1.
pd.merge(df1,df3, on='key', how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,0.0,b,2
4,1.0,b,2
5,6.0,b,2
6,2.0,a,0
7,4.0,a,0
8,5.0,a,0
9,NaN,d,3


In [9]:
#How does an 'inner' join compare?
pd.merge(df1,df3, on='key', how='inner')
# seems to be the default argument...

,data1,key,data2
0,0,b,1
1,0,b,2
2,1,b,1
3,1,b,2
4,6,b,1
5,6,b,2
6,2,a,0
7,4,a,0
8,5,a,0


In [10]:
#How does an 'outer' join compare?  If inner joins are like an intersection of two sets, outer joins are unions.
pd.merge(df1,df3, on='key', how='outer')

,data1,key,data2
0,0.0,b,1.0
1,0.0,b,2.0
2,1.0,b,1.0
3,1.0,b,2.0
4,6.0,b,1.0
5,6.0,b,2.0
6,2.0,a,0.0
7,4.0,a,0.0
8,5.0,a,0.0
9,3.0,c,NaN


In [11]:
#What if the join fields have different names?  No problem - just specify the names.
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'key_2': ['a', 'b', 'b', 'd'],'data2': range(4)})
pd.merge(df4,df5, left_on='key_1', right_on='key_2')

,data1,key_1,data2,key_2
0,0,b,1,b
1,0,b,2,b
2,1,b,1,b
3,1,b,2,b
4,6,b,1,b
5,6,b,2,b
6,2,a,0,a
7,4,a,0,a
8,5,a,0,a


In [12]:
# Here is an example that uses a combination of a data column and an index to merge two dataframes.
df4 = pd.DataFrame({'key_1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df5 = pd.DataFrame({'data2': [4,6,8,10]}, index=['a','b','c','d'])
pd.merge(df4,df5, left_on='key_1', right_index=True)

,data1,key_1,data2
0,0,b,6
1,1,b,6
6,6,b,6
2,2,a,4
4,4,a,4
5,5,a,4
3,3,c,8


## Concatenating

In [13]:
# Concatenating can append rows, or columns, depending on which axis you use. Default is 0
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])
# Since we are concatenating series on axis 0, this creates a longer series, appending each of the three series

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [16]:
# What if we concatenate on axis 1?
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [17]:
# Outer join is the default:
pd.concat([s1, s2, s3], axis=1, join='outer')

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [18]:
# What would an inner join produce?
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


In [19]:
# We need some overlapping values to have the inner join produe non-empty results
s4 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s5 = pd.Series([1, 2, 3], index=['d', 'e', 'f'])
s6 = pd.Series([7, 8, 9, 10], index=['d', 'e', 'f', 'g'])
pd.concat([s4, s5, s6], axis=1, join='outer')
#at least one overlapping set of values in each column

,0,1,2
c,4.0,NaN,NaN
d,5.0,1.0,7.0
e,6.0,2.0,8.0
f,NaN,3.0,9.0
g,NaN,NaN,10.0


In [20]:
# Here is the inner join 
pd.concat([s4, s5, s6], axis=1, join='inner')
# Note that it contains only entries that overlap in all three series.

,0,1,2
d,5,1,7
e,6,2,8


## Reshaping with Hierarchical Indexing

In [21]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [22]:
# Stack pivots the columns into rows, producing a Series with a hierarchical index:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [23]:
# Unstack reverses this process:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


See also the related pivot method

AttributeError: 'Series' object has no attribute 'pivot'

## Data Transformations

In [25]:
# Start with a dataframe containing some duplicate values
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,'k2': [1, 1, 2, 3, 3, 4, 99]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,99


In [26]:
# How to see which rows contain duplicate values
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [27]:
# How to remove duplicate values
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4
6,two,99


In [28]:
#If 99 is a code for missing data, we could replace any such values with NaNs
data2=data['k2'].replace(99,np.nan)

0    1.0
1    1.0
2    2.0
3    3.0
4    3.0
5    4.0
6    NaN
Name: k2, dtype: float64

## Categorization (binning)

In [32]:
# Let's look at how to create categories of data using ranges to bin the data using cut
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
type(cats)

pandas.core.categorical.Categorical

In [33]:
cats.categories

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

In [34]:
cats.codes

array([ 0,  0,  0,  1,  0,  0,  2,  1,  3,  2,  2,  1, -1], dtype=int8)

In [35]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [36]:
# Consistent with mathematical notation for intervals, a parenthesis means that the side is open while the 
#square bracket means it is closed (inclusive). Which side is closed can be changed by passing right=False:
cats = pd.cut(ages, bins, right=False)
print(ages)
print(pd.value_counts(cats))

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32, 101]
[25, 35)     4
[18, 25)     4
[35, 60)     3
[60, 100)    1
dtype: int64


### Removing Outliers

In [40]:
# Start by creating a dataframe with 4 columns of 1,000 random numbers
# We'll use a fixed seed for the random number generator to get repeatable results
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [42]:
# This identifies any values in column 3 with absolute values > 3
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [41]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [43]:
# This identifies all the rows with any column containing absolute values > 3
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [44]:
# Now we can cap the values at -3 to 3 using this:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### Computing Dummy Variables

In [45]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [46]:
# This generates dummy variables for each value of key
# Dummy variables are useful in statistical modeling, to have 0/1 indicator
# variables for the presence of some condition
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [47]:
# This generates dummy variables for each value of key and appends these to the dataframe
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


Notice that we used join instead of merge.  The join method is very similar to merge, but uses indexes to merge, by default.  From the documentation:

http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging
merge is a function in the pandas namespace, and it is also available as a DataFrame instance method, with the calling DataFrame being implicitly considered the left object in the join.

The related DataFrame.join method, uses merge internally for the index-on-index and index-on-column(s) joins, but joins on indexes by default rather than trying to join on common columns (the default behavior for merge). If you are joining on index, you may wish to use DataFrame.join to save yourself some typing

## A bit more: 
1. Filter out records with more than 4 bedrooms
2. Create dummy variables for each bedroom count (e.g. bed_1 would have 1 for rows with 1 bedroom, 0 for others), and merge them with the dataframe
3. Filter sqft < 500 and > 3000
4. Create a set of 5 bins for price and do counts of how many records are in each category